In [1]:
from Tkinter import *
from math import sin, cos, pi, fabs
from numpy import sign
import sys

# Class Sphero - robot object to be created (Done) %run Sphero.ipynb
# Class Walls to be created
# Class world to be created which contains the boundaries, walls (worls is a parent for walls, trajectory, entry and exit points)
# Class Sphero_trajectory - Creating the waypoint trajectory for the robot (child of room)
# Class 1D - Gaussian along the wall to represent the collision (this will become the child of walls)
# Lets create a superior set of features, which contains the boundaries set (1,2,3,4) and the set of walls (1,2,3,4) and it is 
# easy to maintain a set of probability grid and more easy for mapping and SLAM as well.

class world(object):
    """ An artificial world is created for the SLAM problem. The other objects in the world are child classes in the artificial 
    world"""
    def draw(self, at_step):
        print "Object to be drawn at certain point of time:", at_step
        
    def background_draw(self):
        print "Background to be drawn"
        
class boundaries(object):
    
    def __init__(self, world_size, boundary_width):
        self.world_size = world_size
        self.wall_width = wall_width
        self.parray = []   # Consider preallocation for faster memory access
        
    def draw(self,origin,w):
        a = origin[0] 
        b = origin[1]    # to be added in the main loop
        # Outer boundary
        w.create_line(a,b,a+self.world_size[0],b)
        w.create_line(a+self.world_size[0],b, a+self.world_size[0],b+self.world_size[1])
        w.create_line(a,b,a,b+self.world_size[1])
        w.create_line(a,b+self.world_size[1],a+self.world_size[0],b+self.world_size[1])
        
        # Inner boundary
        w.create_line(a+self.wall_width,b+self.wall_width,
                      a-self.wall_width+self.world_size[0],b+self.wall_width)
        w.create_line(a-self.wall_width+self.world_size[0],b+self.wall_width, 
                      a-self.wall_width+self.world_size[0],b-self.wall_width+self.world_size[1])
        w.create_line(a+self.wall_width,b+self.wall_width,
                      a+self.wall_width,b-self.wall_width+self.world_size[1])
        w.create_line(a+self.wall_width,b-self.wall_width+self.world_size[1],
                      a-self.wall_width+self.world_size[0],b-self.wall_width+self.world_size[1])
        
    def boundary_grid(self,origin,w): # Defining the probability grid for the boundary by taking input as the corners
        # Assuming corners are available in the form of a list (given in an sequential fashion)
        a = origin[0]
        b = origin[1]
        d = 100     # Extent or level of discretization 
        
        # Addition of corners in a clockwise fashion
        corner = []
        corner.append([a + self.wall_width, b + self.wall_width])
        corner.append([a-self.wall_width+self.world_size[0],b+self.wall_width])
        corner.append([a-self.wall_width+self.world_size[0],b-self.wall_width+self.world_size[1]])
        corner.append([a+self.wall_width,b-self.wall_width+self.world_size[1]])

        for j in range(len(corner)):
            length = self.length_boundary(corner[j%len(corner)],corner[(j+1)%len(corner)])
            if length[1] == -1:
                print "Error in computing slope - Not a rectilinear world"
                sys.exit()

            for i in range(d):
                self.parray.append([corner[j][0]+sign(corner[(j+1)%len(corner)][0]-corner[j%len(corner)][0])*sin(length[1])*length[0]*i/d,
                                    corner[j][1]+sign(corner[(j+1)%len(corner)][1]-corner[j%len(corner)][1])*cos(length[1])*length[0]*i/d,0])
                w.create_line(self.parray[-1][0],self.parray[-1][1],
                              self.parray[-1][0]+sign(corner[(j+1)%len(corner)][1]-corner[j%len(corner)][1])*self.wall_width,
                              self.parray[-1][1]-sign(corner[(j+1)%len(corner)][0]-corner[j%len(corner)][0])*self.wall_width)
#                 w.create_rectangle(self.parray[-1][0],self.parray[-1][1],
#                                    self.parray[-2][0]+sign(corner[(j+1)%len(corner)][1]-corner[j%len(corner)][1])*self.wall_width,
#                                    self.parray[-2][1]-sign(corner[(j+1)%len(corner)][0]-corner[j%len(corner)][0])*self.wall_width)
            w.create_line(corner[j%len(corner)][0],corner[j%len(corner)][1],
                          corner[j%len(corner)][0]+sign(corner[j%len(corner)][1]-corner[(j-1)%len(corner)][1])*self.wall_width,
                          corner[j%len(corner)][1]-sign(corner[j%len(corner)][0]-corner[(j-1)%len(corner)][0])*self.wall_width)
        return self.parray
    
#     def histogram_distribution(self,collision,parray):
        
        
    def length_boundary(self,cornerA,cornerB):
        
        a = max(fabs(cornerA[0]-cornerB[0]),fabs(cornerA[1]-cornerB[1])) # length of the line
        # To compute the slope of the line
        if cornerA[0]-cornerB[0]==0:
            b = 0
        elif cornerA[1]-cornerB[1]==0:
            b = pi/2
        else: # It is already in clockwise fashion; to report if something goes false
            b = -1
            
        return (a,b)
    
class walls(object):
    def __init__(self,wall_width,location):
        self.warray=[]
        self.wall_width = wall_width # necessary to create variable width blocks
        
    def draw(self,ww,loc,w):
        
        for i in range(len(loc)):
            m = self.slope(loc[i])
            w.create_line(loc[i][0]-m*ww/2,loc[i][1]-(1-m)*ww/2,loc[i][0]+m*ww/2,loc[i][1]+(1-m)*ww/2)
            w.create_line(loc[i][2]-m*ww/2,loc[i][3]-(1-m)*ww/2,loc[i][2]+m*ww/2,loc[i][3]+(1-m)*ww/2)
            w.create_line(loc[i][0]-m*ww/2,loc[i][1]-(1-m)*ww/2,loc[i][2]-m*ww/2,loc[i][3]-(1-m)*ww/2)
            w.create_line(loc[i][0]+m*ww/2,loc[i][1]+(1-m)*ww/2,loc[i][2]+m*ww/2,loc[i][3]+(1-m)*ww/2)
            
    def slope(self,wall):
        ws = wall[:2]
        we = wall[2:]
        if fabs(ws[1]-we[1]) < 1e-3:
            m = 0
        elif fabs(ws[0]-we[0]) < 1e-3:
            m = 1
        else:
            m = -1
            print 'Error: Rectilinear walls are only allowed, check this wall {0}'.format(wall)
            sys.exit()
        return m
    
    def wall_grid(self,ww,loc,w):
        
        d = 100  # Discretization of the grid
        
        for i in range(len(loc)):
            m = self.slope(loc[i])
            ws = loc[i][:2]
            we = loc[i][2:]
            for i in range(d):
                self.warray.append([ws[0]+sign(we[0]-ws[0])*(we[0]-ws[0])*i/d,
                                   ws[1]+sign(we[1]-ws[1])*(we[1]-ws[1])*i/d])
                w.create_line(self.warray[-1][0]+sign(we[1]-ws[1])*ww/2,self.warray[-1][1]+sign(we[0]-ws[0])*ww/2,
                              self.warray[-1][0]-sign(we[1]-ws[1])*ww/2,self.warray[-1][1]-sign(we[0]-ws[0])*ww/2)
                
        return self.warray
        
        
if __name__ =="__main__":

    # Let's create my world
#     origin = (600,200) # Origin of the new world (From the output frame of Tkinter)
    origin = (2,2)
    world_size = (498, 498)   # 750,750
    wall_width = 10
#     wall_location = [[150,150,200,150],[300,300,120,300]]
    
    wall_location =[[origin[0]+(world_size[0])/4,origin[1]+wall_width,origin[0]+(world_size[0])/4,origin[1]+wall_width+(world_size[1])*3/4],
                    [origin[0]+(world_size[0])/2,origin[1]+wall_width+(world_size[1])*1/4,origin[0]+(world_size[0])/2,origin[1]+world_size[1]],
                    [origin[0]+(world_size[0])*3/4,origin[1]+wall_width,origin[0]+(world_size[0])*3/4,origin[1]+wall_width+(world_size[1])*3/4]] 
    
    sri = boundaries(world_size,wall_width*4)
    mw = walls(wall_width,wall_location)
    
    
    master = Tk()
    w = Canvas(master,width=500,height=500) # 2500,2500
    w.pack()
    sri.draw(origin,w)
    mw.draw(wall_width,wall_location,w)
    parray = sri.boundary_grid(origin,w)
    warray = mw.wall_grid(wall_width,wall_location,w)
    mainloop()

In [8]:
from Tkinter import *
from time import sleep
import sys
w = Canvas(Tk(),width=500,height=500)
w.pack()
i=0
w.create_oval(100,0,0,100,fill="red")
mainloop()
for i in range(0,10,1):
    print i
    if i%2==0:
        w.create_rectangle(100,100,20,20,fill="red")
    else:
        w.create_rectangle(100,100,20,20,fill="blue")
    i=i+1
    update_idletasks()
    sleep(2)

0


TclError: invalid command name ".66058440L"

In [26]:
from Tkinter import *
class Foo(Frame):
    def __init__(self,master=None):
#         Frame.__init__(self,master)
        Frame = Tk()
        w = Canvas(Frame,width=500,height=500)
        w.pack()
        self.x=0
        self.id=self.after(1000,self.callme)
        
    def callme(self):
        self.x+=1
        print self.x
#         self.create_oval(0,0,1,1,fill="red")
        self.id=self.after(1000,self.callme)  #can cancel the call by self.after_cancel(self.id)

Gui=Foo()
Gui.mainloop()

AttributeError: Foo instance has no attribute 'tk'